<a href="https://colab.research.google.com/github/git-sathiya/apache-pyspark-by-example-802868/blob/main/Apache_PySpark_by_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apache PySpark by Example

**[June 2023 update]**

I've consolidated all the notebooks for this course into a single notebook. (The course videos will still show individual notebooks)


## Introduction to Google Colab

### Jupyter notebook basics

#### Code cells

In [ ]:
!ls

sample_data


In [ ]:

!pwd

/content


#### Text cells

### Access to the shell

In [ ]:
!ls

sample_data


In [ ]:
!pwd

/content


In [ ]:
!ls -ltrah

total 16K
drwxr-xr-x 4 root root 4.0K May 22 13:22 .config
drwxr-xr-x 1 root root 4.0K May 22 13:23 .
drwxr-xr-x 1 root root 4.0K May 22 13:23 sample_data
drwxr-xr-x 1 root root 4.0K May 23 23:11 ..


In [ ]:
##Import Statements

In [ ]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

## Install Spark

- Google colab recently made some changes which breaks the Spark installation.
- Please use the code below where we install from the pyspark package instead

In [ ]:
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=ec6cf48c63bd80c317a1402e99bc7e1bededbe2d8d17408deca160bf15bd03a0
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
spark

## (02-02) Download Chicago's Reported Crime Data

### Downloading and preprocessing Chicago's Reported Crime Data

In [ ]:
#!ls -l
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

--2024-05-27 06:33:38--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [       <=>          ]   1.77G  2.98MB/s    in 10m 19s 

2024-05-27 06:43:58 (2.93 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1905516889]

total 1860868
-rw-r--r-- 1 root root 1905516889 May 26 10:51 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x 1 root root       4096 May 23 13:25  sample_data


In [ ]:
#!ls -ltrh
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

total 1860868
-rw-r--r-- 1 root root 1905516889 May 26 10:51 reported-crimes.csv
drwxr-xr-x 1 root root       4096 May 23 13:25 sample_data


In [ ]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|     

## (03-03) Schemas

In [ ]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



## (03-04) Working with columns

**Display only the first 5 rows of the column name IUCR**

In [ ]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|1153|
|1153|
|1153|
|0810|
|1153|
+----+
only showing top 5 rows



In [ ]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|1153|
|1153|
|1153|
|0810|
|1153|
+----+
only showing top 5 rows



In [ ]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|1153|
|1153|
|1153|
|0810|
|1153|
+----+
only showing top 5 rows



  **Display only the first 4 rows of the column names Case Number, Date and Arrest**

In [ ]:
rc.select('Case Number','Date','Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA371270|2015-03-18 12:00:00| false|
|   JC212333|2016-05-01 00:25:00| false|
|   JC212935|2014-06-01 00:01:00| false|
|   JC213529|2018-09-01 00:01:00| false|
+-----------+-------------------+------+
only showing top 4 rows



**Add a column with name One, with entries all 1s**

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
rc.withColumn('one',lit(1)).show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|one|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|  

**Remove the column IUCR**

In [ ]:
rc1=rc.drop('IUCR')

In [ ]:
#rc.printSchema()
rc1.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
rc.drop('IUCR').show(4)

+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|        null|        null

In [ ]:
rc.select('ward').show(5)

+----+
|ward|
+----+
|  42|
|  15|
|  21|
|   8|
|  30|
+----+
only showing top 5 rows



## (03-05) Working with rows

**Add the reported crimes for an additional day, 12-Nov-2018, to our dataset.**

In [ ]:
one_day = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))
one_day.count()

4

In [ ]:
one_day.count()

4

In [ ]:
one_day.show()

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13358766|   JH140578|2018-11-12 00:00:00|    008XX E 63RD ST|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           APARTMENT| false| 

In [ ]:
# Adding the 12-Nov-2018 Dataset to dataset rc.

In [ ]:
#rc.count()
#6757087
rc_union = rc.union(one_day).count()

In [ ]:
rc_union = rc.union(one_day).show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|     

In [ ]:
rc_orderBy = rc.union(one_day).orderBy('Date',ascending=False).show(5)

+--------+-----------+-------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13358766|   JH140578|2018-11-12 00:00:00|    008XX E 63RD ST|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           APARTMENT|

In [ ]:
rc.union(one_day).orderBy('Date',ascending=False).show(5)

+--------+-----------+-------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13358766|   JH140578|2018-11-12 00:00:00|    008XX E 63RD ST|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           APARTMENT|

In [ ]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|     

**What are the top 10 number of reported crimes by Primary type, in descending order of occurence?**

In [ ]:
rc.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  46921|
|CRIMINAL SEXUAL A...|   1472|
|            STALKING|   3388|
|PUBLIC PEACE VIOL...|  47785|
|           OBSCENITY|    586|
|               ARSON|  11157|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14422|
|   CRIMINAL TRESPASS| 193372|
|             ASSAULT| 418522|
|LIQUOR LAW VIOLATION|  14068|
| MOTOR VEHICLE THEFT| 314134|
|               THEFT|1418529|
|             BATTERY|1232294|
|             ROBBERY| 255604|
|            HOMICIDE|   9476|
|           RITUALISM|     23|
|    PUBLIC INDECENCY|    161|
| CRIM SEXUAL ASSAULT|  26345|
|   HUMAN TRAFFICKING|     48|
+--------------------+-------+
only showing top 20 rows



In [ ]:
rc.groupBy('Primary Type').orderBY('count',ascending=False).show(10)

AttributeError: 'GroupedData' object has no attribute 'orderBY'

In [ ]:
#For the above error: at the End of any groupBy Operation there should be an aggregation function, prior to using an orderBY

In [ ]:
rc.groupBy('Primary Type').count().orderBy('count',ascending=False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1418529|
|            BATTERY|1232294|
|    CRIMINAL DAMAGE| 771523|
|          NARCOTICS| 711779|
|      OTHER OFFENSE| 419046|
|            ASSAULT| 418522|
|           BURGLARY| 388040|
|MOTOR VEHICLE THEFT| 314134|
| DECEPTIVE PRACTICE| 267365|
|            ROBBERY| 255604|
+-------------------+-------+
only showing top 10 rows



In [ ]:
rc.groupBy('Primary Type').count().orderBy('count',ascending=False).show(5)

+---------------+-------+
|   Primary Type|  count|
+---------------+-------+
|          THEFT|1418529|
|        BATTERY|1232295|
|CRIMINAL DAMAGE| 771523|
|      NARCOTICS| 711779|
|  OTHER OFFENSE| 419046|
+---------------+-------+
only showing top 5 rows



## (03-06) Challenge

**What percentage of reported crimes resulted in an arrest?**

In [ ]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
rc.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [ ]:
rc.groupBy('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
| false|4881736|
|  true|1875351|
+------+-------+



In [ ]:
rc.filter(col('Arrest')=='true').count()/rc.count()
#rc.filter(col('Arrest')=='true').count()/rc.select('Arrest').count()

0.2775383830339908

  **What are the top 3 locations for reported crimes?**

In [ ]:
rc.select('Location').distinct().show()

+--------------------+
|            Location|
+--------------------+
|(41.8022325, -87....|
|(41.70992755, -87...|
|(41.994758989, -8...|
|(41.844015515, -8...|
|(42.019451998, -8...|
|(41.69219741, -87...|
|(41.865523236, -8...|
|(41.891553969, -8...|
|(41.893676531, -8...|
|(41.94980727, -87...|
|(41.854998937, -8...|
|(41.946021295, -8...|
|(41.698454445, -8...|
|(41.759011495, -8...|
|(41.656469322, -8...|
|(41.851322831, -8...|
|(41.857697585, -8...|
|(41.810558994, -8...|
|(41.844262752, -8...|
|(41.800552515, -8...|
+--------------------+
only showing top 20 rows



In [ ]:
rc.groupBy('Location Description').count().show()

+--------------------+------+
|Location Description| count|
+--------------------+------+
|   RAILROAD PROPERTY|    13|
|SCHOOL - PRIVATE ...|    11|
|VEHICLE - COMMERCIAL|    10|
|EXPRESSWAY EMBANK...|     1|
|RESIDENCE - YARD ...|    85|
|POLICE FACILITY/V...| 17599|
|               MOTEL|     5|
|CHA PARKING LOT /...|    19|
|            SIDEWALK|665558|
|PUBLIC GRAMMAR SC...|     1|
|            CAR WASH|  2775|
|   TRUCKING TERMINAL|     1|
|    AIRPORT/AIRCRAFT| 16114|
|            HOSPITAL|     6|
|MEDICAL/DENTAL OF...|  7095|
|    FEDERAL BUILDING|   783|
|             TRAILER|     3|
|SCHOOL, PUBLIC, G...| 28989|
|SPORTS ARENA/STADIUM|  4996|
|               HOUSE|   520|
+--------------------+------+
only showing top 20 rows



In [ ]:
rc.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770637|
|           RESIDENCE|1146424|
|           APARTMENT| 699320|
+--------------------+-------+
only showing top 3 rows



## (04-01) Built-in functions

In [ ]:
from pyspark.sql import functions

In [ ]:
print(dir(functions))

['Any', 'ArrayType', 'Callable', 'Column', 'DataFrame', 'DataType', 'Dict', 'Iterable', 'JVMView', 'List', 'Optional', 'PandasUDFType', 'PySparkTypeError', 'PySparkValueError', 'PythonEvalType', 'SparkContext', 'StringType', 'StructType', 'TYPE_CHECKING', 'Tuple', 'Union', 'UserDefinedFunction', 'ValuesView', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_py_udf', '_from_numpy_type', '_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_columns', '_invoke_function_over_seq_of_columns', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_append', 'array_compact', 'array_contains', 'array_distinct', 'array_except', 'ar

### String functions

**Display the Primary Type column in lower and upper characters, and the first 4 characters of the column**

In [ ]:
from pyspark.sql.functions import lower,upper,substring

In [ ]:
rc.select('Primary Type',lower('Primary Type'),upper('Primary Type'),substring('Primary Type',1,4)).show(5)

+--------------------+--------------------+--------------------+-----------------------------+
|        Primary Type| lower(Primary Type)| upper(Primary Type)|substring(Primary Type, 1, 4)|
+--------------------+--------------------+--------------------+-----------------------------+
|             ASSAULT|             assault|             ASSAULT|                         ASSA|
|               THEFT|               theft|               THEFT|                         THEF|
|OFFENSE INVOLVING...|offense involving...|OFFENSE INVOLVING...|                         OFFE|
|OFFENSE INVOLVING...|offense involving...|OFFENSE INVOLVING...|                         OFFE|
|OFFENSE INVOLVING...|offense involving...|OFFENSE INVOLVING...|                         OFFE|
+--------------------+--------------------+--------------------+-----------------------------+
only showing top 5 rows



### Numeric functions


**Show the oldest date and the most recent date**

In [ ]:
from pyspark.sql.functions import min,max

In [ ]:
# Both of the Syntax works

#rc.select(min('date'),max('date')).show()
rc.select(min(col('Date')),max(col('Date'))).show()

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2001-01-01 00:00:00|2018-11-11 00:00:00|
+-------------------+-------------------+



### Date

**What is 3 days earlier that the oldest date and 3 days later than the most recent date?**

In [ ]:
from pyspark.sql.functions import date_add,date_sub

In [ ]:
  #rc.select( date_sub(min(col('Date')),3),date_add(max(col('Date')),3) ).show()
rc.select(date_sub(min(col('Date')),3), date_add(max(col('Date')),3)).show()

+----------------------+----------------------+
|date_sub(min(Date), 3)|date_add(max(Date), 3)|
+----------------------+----------------------+
|            2000-12-29|            2018-11-14|
+----------------------+----------------------+



In [ ]:
df=
df.select(to_date(lit('2019-12-25')),to_date(lit('2019-25-15'))).show(1)

NameError: name 'df' is not defined

## (04-02) Working with dates

In [ ]:
from pyspark.sql.functions import to_date, to_timestamp, lit

df = spark.createDataFrame([('2019-12-25 13:30:00',)],['Christmas'])
df.show(1)

+-------------------+
|          Christmas|
+-------------------+
|2019-12-25 13:30:00|
+-------------------+



  **2019-12-25 13:30:00**

In [ ]:
df.select(to_date(col('christmas'),'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('christmas'),'yyyy-MM-dd HH:mm:ss')).show(1)

+---------------------------------------+--------------------------------------------+
|to_date(christmas, yyyy-MM-dd HH:mm:ss)|to_timestamp(christmas, yyyy-MM-dd HH:mm:ss)|
+---------------------------------------+--------------------------------------------+
|                             2019-12-25|                         2019-12-25 13:30:00|
+---------------------------------------+--------------------------------------------+



**25/Dec/2019 13:30:00**

In [ ]:
df=spark.createDataFrame([('25/Dec/2019 13:30:00',)],['Christmas'])
df.show(1)

+--------------------+
|           Christmas|
+--------------------+
|25/Dec/2019 13:30:00|
+--------------------+



In [ ]:
df.select(to_date(col('christmas'),'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('christmas'),'dd/MMM/yyyy HH:mm:ss')).show(1)


+----------------------------------------+---------------------------------------------+
|to_date(christmas, dd/MMM/yyyy HH:mm:ss)|to_timestamp(christmas, dd/MMM/yyyy HH:mm:ss)|
+----------------------------------------+---------------------------------------------+
|                              2019-12-25|                          2019-12-25 13:30:00|
+----------------------------------------+---------------------------------------------+



**12/25/2019 01:30:00 PM**

In [ ]:
df=spark.createDataFrame([('12/25/2019 01:30:00 PM' ,)],['Christmas'])
df.show(truncate=False)


+----------------------+
|Christmas             |
+----------------------+
|12/25/2019 01:30:00 PM|
+----------------------+



In [ ]:
#set spark.sql.legacy.timeParserPolicy = "LEGACY"
df.select('Christmas',to_date(col('christmas'),'MM/dd/yyyy HH:mm:ss a'), to_timestamp(col('christmas'),'MM/dd/yyyy HH:mm:ss a')).show(truncate=False)


+----------------------+-----------------------------------------+----------------------------------------------+
|Christmas             |to_date(christmas, MM/dd/yyyy HH:mm:ss a)|to_timestamp(christmas, MM/dd/yyyy HH:mm:ss a)|
+----------------------+-----------------------------------------+----------------------------------------------+
|12/25/2019 01:30:00 PM|null                                     |null                                          |
+----------------------+-----------------------------------------+----------------------------------------------+



## (04-03) Joins

**Download police station data**

In [ ]:
#!wget -O police-station.csv https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data
!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2024-05-27 11:47:21--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-station.csv’

police-station.csv      [ <=>                ]   5.57K  --.-KB/s    in 0s      

2024-05-27 11:47:21 (1.56 GB/s) - ‘police-station.csv’ saved [5699]



In [ ]:
!ls

police-station.csv  reported-crimes.csv  sample_data


In [ ]:
#ps=spark.open.csv('police-station.csv',header=True)
ps= spark.read.csv('police-station.csv',header=True)

In [ ]:
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

In [ ]:
rc.cache()
rc.count()

6757096

**The reported crimes dataset has only the district number. Add the district name by joining with the police station dataset**

In [ ]:
ps.select(col('District')).distinct().show(30)


+--------------------+
|            District|
+--------------------+
|                   7|
|                  15|
|                  11|
|                   3|
|                   8|
|                  22|
|                  16|
|                   5|
|                  18|
|                  17|
|                   6|
|                  19|
|                  25|
|        Headquarters|
|                  24|
|                   9|
|                   1|
|                  20|
|                  10|
|                   4|
|                  12|
|                  14|
|                   2|
|",Chicago,IL,6060...|
+--------------------+



In [ ]:
rc.select('District').distinct().show(30)

+--------+
|District|
+--------+
|     009|
|     012|
|     024|
|    null|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     003|
|     005|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
|     004|
|     017|
|     007|
|     002|
|     021|
+--------+



In [ ]:
from pyspark.sql.functions import lpad
help(lpad)


In [ ]:
ps.select(lpad(col('District'),3,'0')).show()

In [ ]:
ps =ps.withColumn('Format_Distinct',lpad(col('District'),3,'0'))
ps.show()

In [ ]:
rc.join (ps,rc.District ==ps.Format_district, 'left_outer').show()

In [ ]:
ps.columns

In [ ]:
rc.join (ps,rc.District ==ps.Format_district, 'left_outer').drop(<list of columns to be dropped>).show()

## (04-05) Challenge questions

**What is the most frequently reported non-criminal activity?**

**Using a bar chart, plot which day of the week has the most number of reported crime.**

## (05-01) RDDs setup

**How many police stations are there?**

**Display the District ID, District name, Address and Zip for the police station with District ID 7**



**Police stations 10 and 11 are geographically close to each other. Display the District ID, District name, address and zip code**